# OpenAI Assistants - Building Agentic RAG with the Function Calling, Retrieval, and Code Interpreter Tools

Today we'll explore using OpenAI's Python SDK to create, manage, and use the OpenAI Assistant API!

## Dependencies

We'll start, as we usually do, with some dependiencies and our API key!

In [116]:
!pip install -qU openai

In [118]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

OpenAI API Key: ········


## Simple Assistant

Let's create a simple Assistant to understand more about how the API works to start!

### OpenAI Client

At the core of the OpenAI Python SDK is the Client!

> NOTE: For ease of use, we'll start with the synchronous `OpenAI()`. OpenAI does provide an `AsyncOpenAI()` that you could leverage as well!

In [119]:
from openai import OpenAI

client = OpenAI()

### Creating An Assistant

Leveraging what we know about the OpenAI API from previous sessions - we're going to start by simply initializing an Assistant.

Before we begin, we need to think about a few customization options we have:

- `name` - Straight forward enough, this is what our Assistant's name will be
- `instructions` - similar to a system message, but applied at an Assistant level, this is how we can guide the Assistant's tone, behaviour, functionality, and more!
- `model` - this will allow us to choose which model we would prefer to use for our Assistant

Let's start by setting some instructions for our Assistant.



In [120]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown Fill out the fields below to add your Assistant's name, instructions, and desired model!

name = "NAME HERE" # @param {type: "string"}
instructions = "INSTRUCTIONS HERE" # @param {type: "string"}
model = "gpt-3.5-turbo" # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4"]

### Initialize Assistant

Now that we have our desired name, instruction, and model - we can initialize our Assistant!

In [121]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)

Let's examine our `assistant` object and see what we find!

In [122]:
assistant

Assistant(id='asst_RidbG851hukUbQpQ4fOEjvg7', created_at=1713671254, description=None, file_ids=[], instructions='INSTRUCTIONS HERE', metadata={}, model='gpt-3.5-turbo', name='NAME HERE', object='assistant', tools=[], top_p=1.0, temperature=1.0, response_format='auto')

There are a number of useful parameters here, but we'll call out a few:

- `id` - since we may have multiple Assistant's, knowing which Assistant we're interacting with will help us ensure the desired user experience!
- `description` - A natrual language description of our Assistant could help others understand what it's supposed to do!
- `file_ids` - if we wanted to use the Retrieval tool, this would let us know what files we had given our Assistant

### Creating a Thread

Behind the scenes our Assistant is powered by the idea of "threads".

You can think of threads as individual conversations that interact with the Assistant.

Let's create a thread now!

In [123]:
thread = client.beta.threads.create()

Let's look at our `thread` object.

In [124]:
thread

Thread(id='thread_aMgdc8fuBU2Oh9dovNAOvH3u', created_at=1713671255, metadata={}, object='thread')

Notice some key attributes:

- `id` - since each Thread is like a conversation, we need some way to specify which thread we're dealing with when interacting with them
- `tool_resources` - this will become more relevant as we add tools since we'll need a way to verify which tools we have access to when interacting with our Assistant

### Adding Messages to Our Thread

Now that we have our Thread (or conversation) we can start adding messages to it!

Let's add a simple message that asks about how our Assistant is feeling.

Notice the parameters we're leveraging:

- `thread_id` - since each Thread is like a conversation, we need some way to address a specific conversation. We can use `thread.id` to do this.
- `role` - similar to when we used our chat completions endpoint, this parameter specifies who the message is coming from. You can leverage this in the same ways you would through the chat completions endpoint.
- `content` - this is where we can place the actual text our Assistant will interact with

> NOTE: Feel free to substitute a relevant message based on the Assistant you created

In [125]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"How old are you?"
)

Again, let's examine our `message` object!

In [126]:
message

Message(id='msg_ucJXSPuCaeepCMdz1f6eDFfp', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How old are you?'), type='text')], created_at=1713671256, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_aMgdc8fuBU2Oh9dovNAOvH3u')

### Running Our Thread

Now that we have an Assistant, and we've given that Assistant a Thread, and we've added a Message to that Thread - we're ready to run our Assistant!

Notice that this process lets us add (potentially) multiple messages to our Assistant. We can leverage that behaviour for few/many-shot examples, and more!

In [127]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown We can also override the Assistant's instructions when we run a thread.

# @markdown Use one of the [Prompt Principles for Instruction](https://arxiv.org/pdf/2312.16171v1.pdf) to improve the likeliehood of a correct or valuable response from your Assistant.

additional_instructions = "ADDITIONAL INSTRUCTIONS" # @param {type: "string"}

Let's run our Thread!

In [128]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=instructions + " " + additional_instructions
)

Now that we've run our thread, let's look at the object!

In [129]:
run

Run(id='run_kXOH7S8HGuNZ3T3ZCXIqFNzw', assistant_id='asst_RidbG851hukUbQpQ4fOEjvg7', cancelled_at=None, completed_at=None, created_at=1713671257, expires_at=1713671857, failed_at=None, file_ids=[], incomplete_details=None, instructions='INSTRUCTIONS HERE ADDITIONAL INSTRUCTIONS', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_aMgdc8fuBU2Oh9dovNAOvH3u', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0)

Notice we have access to a few very powerful parameters in this `run` object.

- `completed_at` - this will help us determine when we can expect to retrieve a response
- `failed_at` - this can highlight any issues our run ran into
- `status` - is another way we can understand how the flow is going

### Retrieving Our Run

Now that we've created our run, let's retrieve it.

We're going to wrap this in a simple loop to make sure we're not retrieving it too early.

In [130]:
import time

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [131]:
print(run.status)

completed


Now that our run is completed - we can retieve the messages from our thread!

Notice that our run helps us understand how things are going - but it isn't where we're going to find our responses or messages. Those are added on the backend into our thread.

This leads to a simple, but important, flow:

1. We add messages to a thread.
2. We create a run on that thread.
3. We wait until the run is finished.
4. We check our thread for the new messages.

### Checking Our Thread

Now we can get a list of messages from our thread!

In [132]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [133]:
messages.data[0]

Message(id='msg_TcxDTfKWB2eyrPeUgVRIeeIe', assistant_id='asst_RidbG851hukUbQpQ4fOEjvg7', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I am a chatbot AI and do not have an age in the traditional sense. How can I assist you today?'), type='text')], created_at=1713671258, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_kXOH7S8HGuNZ3T3ZCXIqFNzw', status=None, thread_id='thread_aMgdc8fuBU2Oh9dovNAOvH3u')

## Adding Tools

Now that we have an understanding of how Assistant works, we can start thinking about adding tools.

We'll go through 3 separate tools and explore how we can leverage them!

Let's start with the most familiar tool - the Retriever!


### Creating an Assistant with the Retriever Tool

The first thing we'll want to do is create an assistant with the Retriever tool.

This is also going to require some data. We'll provided data - but you're very much encouraged to use your own files to explore how the Assistant works for your use case.

#### Collect and Add Data

First, we need some data. Second, we need to add the data to our Assistant!

Let's start with grabbing some data!

In [134]:
!curl https://github.com/xiaopangxia/TCM-Ancient-Books/blob/master/000-%E7%A5%9E%E5%86%9C%E6%9C%AC%E8%8D%89%E7%BB%8F.txt -o 本草纲目.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  500k    0  500k    0     0   489k      0 --:--:--  0:00:01 --:--:--  490k


Now we can upload our file!

Pay attention to [this](https://platform.openai.com/docs/assistants/tools/supported-files) documentation to see what kinds of files can be uploaded.

> NOTE: Per the OpenAI [docs](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) The maximum file size is 512 MB and no more than 2,000,000 tokens (computed automatically when you attach a file)

In [135]:
file_reference = client.files.create(
  file=open("本草纲目.html", "rb"),
  purpose='assistants'
)

Let's look at what our `file_reference` contains!

In [136]:
file_reference

FileObject(id='file-ONIr6Rvp1U8AaCAlPhDiJqE2', bytes=512860, created_at=1713671262, filename='本草纲目.html', object='file', purpose='assistants', status='processed', status_details=None)

#### Create and Use Assistant

Now that we have our file - we can attach it to an Assistant, and we can give that Assistant the ability to use it for retrieval through the Retrieval tool!

> NOTE: Please pay attention to [pricing](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) and don't forget to delete your files when you're done!

In [137]:
assistant = client.beta.assistants.create(
  name=name + "+ Retrieval",
  instructions=instructions,
  model=model,
  tools=[{"type": "retrieval"}],
  file_ids=[file_reference.id]
)

Let's try submitting a message to our Assistant and seeing what kind of answer we get!

We'll outline the steps needed to do this in full:

1. Create an Assistant
2. Create a Thread
3. Add Messages to that Thread
4. Create a Run on that Thread
5. Wait for Run to Complete
6. Collect Messages from the Thread

Let's do that below!

In [138]:
# Create a Thread
thread = client.beta.threads.create()

# Add Messages to that Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"云母有什么作用？什么季节适合服务用？能治疗什么症状？"
)

# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress


Let's look at the final result!

In [139]:
messages

SyncCursorPage[Message](data=[Message(id='msg_zADSiorHkCLH3WfxxfjNjIzl', assistant_id='asst_3A3EPDa5xgdrXzrGIMfv04Ml', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='根据上传的文件，云母是一种常见的药材，具有清热燥湿、外用消肿的功效。关于云母的具体作用和用法，文件中提到：\n\n- 云母有利尿的作用，可以在夏秋季节使用。\n- 云母可以治疗水肿、气胀、惊悸等症状。'), type='text')], created_at=1713671274, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_w6dXO66XWQLH4K3YqOn02eci', status=None, thread_id='thread_knmKEec2YPDsnOFqHtNP7cuK'), Message(id='msg_P7bnpuYM74QGhwQJeqxhiWwU', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='云母有什么作用？什么季节适合服务用？能治疗什么症状？'), type='text')], created_at=1713671272, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_knmKEec2YPDsnOFqHtNP7cuK')], object='list', first_id='msg_zADSiorHkCLH3WfxxfjNjIzl', las

Let's do some clean up to make sure we're not being charged anything extra by deleting our resources.

In [140]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

### Creating an Assistant with the Code Interpreter Tool

Now that we've explored the Retrieval Tool - let's try the Code Interpreter tool!

The process will be almost exactly the same - but we can explore a different query, and we'll add our file at the Message level!

In [141]:
assistant = client.beta.assistants.create(
  name=name + "+ Code Interpreter",
  instructions=instructions,
  model=model,
  tools=[{"type": "code_interpreter"}],
  file_ids=[file_reference.id]
)

In the following example, we'll also see how we can package the Thread creation with the Message adding step!

> NOTE: Files added at the message/thread level will not be available to the Assistant outside of that Thread.

In [142]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What kind of file is this?",
      "file_ids": [file_reference.id]
    }
  ]
)

> NOTE: Remember that we create runs at the *thread* level - and so don't need the message object to continue.

In [143]:
# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress


We can check the specific steps that the Code Interpreter ran to figure out what steps the Assistant took!

In [144]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [145]:
for step in run_steps.data:
  print(step.step_details)

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_IUXISOXJIbyoKSJexgb1hxxU'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_Aek2b0d04Zm7CFJsk1y3VRLx', code_interpreter=CodeInterpreter(input="# Display the first few bytes of the file to identify its type\nwith open(file_path, 'rb') as f:\n    first_few_bytes = f.read(50)\n\nfirst_few_bytes", outputs=[CodeInterpreterOutputLogs(logs='b\'\\n\\n\\n\\n\\n\\n<!DOCTYPE html>\\n<html\\n  lang="en"\\n  \\n  data-\'', type='logs')]), type='code_interpreter')], type='tool_calls')
MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_RS0fwe2zzkdug56HfBPooyLa'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_Kg0r7uusqvLRmhwxTx7QL1Bh', code_interpreter=CodeInterpreter(input='# Try to infer the file type based on file extension\nimport os\n\nfile_extension = os.path.splitext(file_path)[-1]\n\nfile_extension', outputs

In [146]:
messages

SyncCursorPage[Message](data=[Message(id='msg_IUXISOXJIbyoKSJexgb1hxxU', assistant_id='asst_OHvr0t1NTHUtMCRiLn4NPt0D', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Based on the content, it appears that the file is an HTML file.'), type='text')], created_at=1713671303, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_KVI7JB9mCNtIbY6i1YZGVSto', status=None, thread_id='thread_aMkwpRic2F1IfKKq0qLcAn1o'), Message(id='msg_RS0fwe2zzkdug56HfBPooyLa', assistant_id='asst_OHvr0t1NTHUtMCRiLn4NPt0D', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The file extension could not be extracted from the file path. Let me display the first few bytes of the file to see if we can identify the file type based on the content.'), type='text')], created_at=1713671299, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assis

In [147]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

And there you go!

We've fit our Assistant with an awesome Code Interpreter that lets our Assistant run code on our provided files!

### Creating an Assistant with a Function Calling Tool

Let's finally create an Assistant that utilizes the Function Calling API.

We'll start by creating a function that we wish to be called.

We'll utilize DuckDuckGo search to allow our Assistant to have the most up to date information!

In [148]:
!pip install -qU duckduckgo_search

In [149]:
from duckduckgo_search import DDGS

def duckduckgo_search(query):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, max_results=5)]
    return "\n".join(result["body"] for result in results)

Let's test our function to make sure it behaves as we expect it to.

In [150]:
duckduckgo_search("Who is the current captain of the Winnipeg Jets?")

"Lowry will follow Andrew Ladd and Blake Wheeler to serve as the third captain of the new Winnipeg Jets franchise. - Sep 12, 2023. After a season without a captain, the Winnipeg Jets have named ...\nLowry has been a part of the Jets organization since June 25, 2011, when he was selected in the third round, 67 th overall, after putting up 37 points in 36 games with the Swift Current Broncos of ...\nAdam Lowry, who has been a Jet since 2011 when he was drafted 67th overall, is the new captain of the NHL team — its third since relocating to Winnipeg from Atlanta in 2011. Andrew Ladd served ...\nAdam Lowry was named captain of the Winnipeg Jets on Tuesday. The 30-year-old forward was selected by the Jets in the third round (No. 67) of the 2011 NHL Draft and has played his entire nine ...\nThe Winnipeg Jets have officially made the decision on which player will wear the captain's 'C' for the 2023-24 season, and hopefully onward. That player is 30-year-old centre Adam Lowry."

Now we need to express how our function works in a way that is compatible with the OpenAI Function Calling API.

We'll want to provide a `JSON` object that includes what parameters we have, how to call them, and a short natural language description.

In [151]:
ddg_function = {
    "name" : "duckduckgo_search",
    "description" : "Answer non-technical questions. ",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "query" : {
                "type:" : "string",
                "description" : "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"
            }
        },
        "required" : ["query"]
    }
}

#### ❓Question

Why does the description key-value pair matter?

#### Answer
1. Clear for developers to read and maintain
2. Easier for models to read and execute
3. Following the same standard is convenient for automations between different applications

Now when we create our Assistant - we'll want to include the function description as a tool using the following format.

In [152]:
assistant = client.beta.assistants.create(
    name=name + " + Function Calling API",
    instructions=instructions,
    tools=[
        {"type": "function",
         "function" : ddg_function
        }
    ],
    model=model
)

We need to make a few modifications to our Assistant to include the ability to make calls to our local function and pass the results back to our Assistant for further generation.

In [153]:
import json

def wait_for_run_completion(thread_id, run_id):
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

def submit_tool_outputs(thread_id, run_id, tools_to_call):
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        if function_name == "duckduckgo_search":
            print("Consulting Duck Duck Go...")
            output = duckduckgo_search(query=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    print(tool_output_array)

    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_output_array
    )

def print_messages_from_thread(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for msg in messages:
        print(f"{msg.role}: {msg.content[0].text.value}")

def use_assistant(query, assistant_id, thread_id=None):
  thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content=query,
  )

  print("Creating Assistant ")

  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )

  print("Querying OpenAI Assistant Thread.")

  run = wait_for_run_completion(thread.id, run.id)

  if run.status == 'requires_action':
    run = submit_tool_outputs(thread.id, run.id, run.required_action.submit_tool_outputs.tool_calls)
    run = wait_for_run_completion(thread.id, run.id)

  print_messages_from_thread(thread.id)

  return thread.id

#### ❓ Question

Outline, in simple terms, what the `use_assistant` helper function is doing.

#### Answer
1. Create Threads.
2. Add Messages.
3. Run the Assistant on the Thread.
4. Wait for the run status and pick up tools to run.
5. Wait for status to see further operations need to take.
6. Generate logs.

In [154]:
use_assistant("Who is the current Captain of the Winnipeg Jets?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_9vZ7K2rsGlSoyw78csJlTHZD', 'output': 'Lowry has been a part of the Jets organization since June 25, 2011, when he was selected in the third round, 67 th overall, after putting up 37 points in 36 games with the Swift Current Broncos of ...\nLowry will follow Andrew Ladd and Blake Wheeler to serve as the third captain of the new Winnipeg Jets franchise. - Sep 12, 2023. After a season without a captain, the Winnipeg Jets have named ...\nAdam Lowry, who has been a Jet since 2011 when he was drafted 67th overall, is the new captain of the NHL team — its third since relocating to Winnipeg from Atlanta in 2011. Andrew Ladd served ...\nWinnipeg Jets Captains. Team Names: Winnipeg Jets, Atlanta Thrashers. Seasons: 24 (1999-00 to 2023-24) NHL Playoff Appearances: 7. NHL Championships: 0 (0 Stanley Cup) ... Current Summary/Standings, Current Schedule/Results

'thread_5ADDiDWxHM4LX2h3wiTzRRUy'

## Wrapping it All Together

Now we can create an Assistant with all of the available tools and see how it responds to various queries!

In [155]:
assistant = client.beta.assistants.create(
    name=name + " + All Tools",
    instructions=instructions,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function" : ddg_function}
    ],
    model=model,
    file_ids=[file_reference.id],
)

In [168]:
use_assistant("Who is the current Captain of the Winnipeg Jets?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The current Captain of the Winnipeg Jets is Blake Wheeler【7:0†source】.
user: Who is the current Captain of the Winnipeg Jets?


'thread_3GRdihHsQ1I2IDCzvuCx82d2'

In [167]:
use_assistant("Who is the author of the supplied file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: completed
assistant: The author of the supplied file is "本草纲目"【5†source】.
user: Who is the author of the supplied file?


'thread_CCko4i7dNq9nWs97XrUiu2aq'

In [173]:
use_assistant("How many bytes is the provided file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: completed
assistant: The provided file is 512,860 bytes in size.
user: How many bytes is the provided file?


'thread_DpqSS3UUEUuRrHZ3ASVNux4D'

#### ❓ Question

Notice that our response can go through multiple paths, given that:

What is "deciding" to use the tool?

#### Answer

1. Assistant Configurations: which tool is available.
2. User Query Analysis: the model will determine with its obejectves.
3. New responses from the answer will be the new queries.

### Adding JSON Mode for More Agentic Behaviour

Finally, we have the ability to select tools - all we need to do now is set up a process to allow us to create some kind of loop and make decisions about whether or not the response is complete or not.

We'll leverage the OpenAI completions end-endpoint with JSON mode to let us understand when we've adequately answered our user's question!

In [159]:
completed_template = \
"""
Does this response adequately answer the user's query?

Please return your response in JSON format - with key: "completed" and either True (if completed) or False (if not completed)

User Query:
{query}

Assistant Response:
{response}
"""

def is_complete(query, response):
  completed_response = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": completed_template.format(query=query, response=response),
          }
      ],
      model=model,
      response_format={"type" : "json_object"}
  )

  return completed_response

In [170]:
query = "How many bytes is the provided file?"

thread_id_for_response = use_assistant(query, assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The size of the provided file is 512,860 bytes.
user: How many bytes is the provided file?


Now we can observe JSON mode in action!

In [171]:
messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
response = messages.data[0].content[0].text.value
completed_flag = json.loads(is_complete(query, response).choices[0].message.content)

In [172]:
completed_flag

{'completed': True}

## 🚧 BONUS CHALLENGE 🚧:

Use the components we've constructed so far to build a loop that lets us continue to query the Assistant if the response is not completed!

In [181]:
### YOUR CODE HERE
max_attempts = 3
attempts = 0
completed_flag = False

while attempts < max_attempts and not completed_flag:
    thread_id_for_response = use_assistant(query, assistant.id)
    messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
    response = messages.data[0].content[0].text.value
    completed_flag = json.loads(is_complete(query, response).choices[0].message.content)['completed']

    if completed_flag:
        print("Query completed successfully.")
        break
    else:
        print("Response incomplete, refining the query.")
    attempts +=1

if completed_flag:
    print("Failed to complete the query after maximum attempts.")


Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: completed
assistant: Of course! I would be happy to help. Could you please provide me with more specific details or context about what you need assistance with? This will help me provide you with the most relevant and accurate information.
user: Could you provide more details?
Response incomplete, refining the query.
Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: completed
assistant: Of course! Can you please specify what you would like more details on? This will help me provide you with the information you need.
user: Could you provide more details?
Response incomplete, refining the query.
Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: completed
assistant: Of course! Can you please specify what you need more details about? This will help me provide you with the specific information you are lookin

# Make Sure You Delete Resources

Make sure you delete all the resources you created!

This function will help you do so!

In [179]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)